<a href="https://colab.research.google.com/github/xainasx/Web/blob/main/Web_crawl_for_Coventry_Uni_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install schedule
import csv
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.robotparser import RobotFileParser
from urllib.parse import urljoin
import time
import schedule
import pandas as pd
import re
from collections import defaultdict
from nltk.stem import SnowballStemmer

In [4]:
base = "https://pureportal.coventry.ac.uk"
pub_url = urljoin(base, "/en/organisations/research-centre-for-computational-science-and-mathematical-modell/publications/")

In [5]:
robot = RobotFileParser()
robot.set_url(urljoin(base, "robots.txt"))
robot.read()
delay = robot.crawl_delay("*")

In [6]:
def can_crawl(url):
    return robot.can_fetch("*", url)

In [7]:
def crawl():
    next_url = pub_url
    data = []
    while next_url and can_crawl(next_url):
        response = requests.get(next_url)
        parse = BeautifulSoup(response.text, 'html.parser')
        for pub in parse.find_all("div", class_="result-container"):
            title = pub.find("h3", class_="title").text
            link = pub.find("a", class_="link")['href']
            date = pub.find("span", class_="date").text
            authors = [a.text for a in pub.find_all("a", class_="link person")]
            data.append({'title': title, 'link': link, 'date': date, 'authors': authors})

        next_page = parse.find('a', class_='nextLink', href=True)
        next_url = urljoin(base, next_page['href']) if next_page else None

    with open('publicat.csv', mode='w', newline='', encoding='utf-8') as output_file:
        writer = csv.writer(output_file)
        writer.writerow(['title', 'link', 'date', 'authors'])
        for record in data:
            writer.writerow(record.values())

In [8]:
run = input("Do you want to run the crawler? Press 'Y'.\nIf you want to schedule weekly updates, press '0'.\n").lower()
if run == 'y':
    crawl()
if run == "0":
    schedule.every().week.do(crawl)
    while True:
        schedule.run_pending()
        time.sleep(delay*10)

Do you want to run the crawler? Press 'Y'.
If you want to schedule weekly updates, press '0'.
y


In [9]:
os.path.isfile('publicat.csv')

True

In [10]:
df = pd.read_csv("publicat.csv")
df.head()

,title,link,date,authors


In [11]:

# Define the preprocessing function
def preprocess_text(text):
    # Convert all text to lowercase
    text = text.lower()
    # Remove non-alphanumeric characters
    text = re.sub(r'\W+', ' ', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text.strip())
    # Remove stop words
    stop_words = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
                  'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with'}
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    # Stem the remaining words
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in filtered_words]
    text = ' '.join(stemmed_words)
    return text

In [12]:
df['processed_title'] = df['title'].apply(preprocess_text)

In [13]:
inverted_index = defaultdict(set)
for i, row in df.iterrows():
    title = row['processed_title']
    words = title.split()
    for word in words:
        inverted_index[word].add(i)

In [14]:
user_query = input('Enter yoursearch input: ')
processed_query = preprocess_text(user_query)

# Search for matching documents
matching_docs = set()
for word in processed_query.split():
    if word in inverted_index:
        matching_docs.update(inverted_index[word])
if not matching_docs:
    print('No results found.')
else:
    # Display the matched documents
    print('Matched documents:')
    for i in matching_docs:
        doc = df.loc[i]
        print(f"- {doc['title']} \n {doc['authors']} \n({doc['date']})\n   {doc['link']}")
        print("--------------------------------------------------------------------------------------")

Enter yoursearch input: Python
No results found.
